In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import timeit
import datetime
import time
import pprint
import itertools
import pickle
import sklearn
import dask
import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')
import fastparquet
seed = 3
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import h5py
import collections
import dask.dataframe as dd

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [24]:
nt = fastparquet.ParquetFile("DataStore/2018-08/newterms_lead_follow_0910.parq").to_pandas().drop_duplicates()
print(len(nt))

import math

def sim_bin(x, a):
    try:
        return round(round(x / a) * a, -int(math.floor(math.log10(a))))
    except:
        return np.nan

nt["bin_sim_docvecs"] = nt["sim_docvecs"].apply(lambda x: sim_bin(x, 0.1))
nt.loc[nt["bin_sim_docvecs"] <= -0.1, "bin_sim_docvecs"] = -0.1
nt.loc[nt["bin_sim_docvecs"] >= 0.7, "bin_sim_docvecs"] = 0.7

nt = nt.loc[nt["term"].isin(nt["term"].value_counts()[:20].index)]

n2 = nt.loc[(nt["tp_primclass"] == nt["op_primclass"]) & (nt["common_pat_inv"] == False)]
print(len(n2))

5609234
79754


In [26]:
sp.stats.ttest_ind(n2.loc[n2["inv_msa_match"] == True, "sim_docvecs"].values,
                  n2.loc[n2["inv_msa_match"] == False, "sim_docvecs"].values,
                  nan_policy="omit")

Ttest_indResult(statistic=14.56362397554488, pvalue=5.515862750083704e-48)

In [31]:
n2[["bin_sim_docvecs", "inv_msa_match", "direct_cite"]].groupby(["bin_sim_docvecs", "inv_msa_match"]).mean()

direct_cite
bin_sim_docvecs inv_msa_match             
-0.1            False             0.000000
                True              0.000000
 0.0            False             0.000140
                True              0.000000
 0.1            False             0.000833
                True              0.001357
 0.2            False             0.000951
                True              0.001720
 0.3            False             0.002099
                True              0.013265
 0.4            False             0.003075
                True              0.012111
 0.5            False             0.004812
                True              0.030837
 0.6            False             0.010558
                True              0.047619
 0.7            False             0.020134
                True              0.000000

In [34]:
n2[["bin_sim_docvecs", "inv_msa_match", "num_common_cites"]].groupby(["bin_sim_docvecs", "inv_msa_match"]).mean()

num_common_cites
bin_sim_docvecs inv_msa_match                  
-0.1            False                  0.004310
                True                   0.000000
 0.0            False                  0.003774
                True                   0.015038
 0.1            False                  0.007262
                True                   0.012212
 0.2            False                  0.011143
                True                   0.030954
 0.3            False                  0.026985
                True                   0.058163
 0.4            False                  0.045227
                True                   0.060554
 0.5            False                  0.080192
                True                   0.158590
 0.6            False                  0.123680
                True                   0.317460
 0.7            False                  0.208054
                True                   0.285714

In [35]:
n2["num_common_cites"].value_counts()

0     78859
1       593
2       149
3        49
4        40
8        17
10       13
5         9
7         7
6         7
9         6
11        2
17        1
13        1
12        1
Name: num_common_cites, dtype: int64

In [25]:
n2[["inv_msa_match", "sim_docvecs", "year_diff"]].groupby(["inv_msa_match", "year_diff"]).mean()

sim_docvecs
inv_msa_match year_diff             
False         1.0           0.211301
              2.0           0.216140
              3.0           0.196004
              4.0           0.208861
              5.0           0.205707
              6.0           0.206917
              7.0           0.208697
              8.0           0.209278
              9.0           0.209811
True          1.0           0.244696
              2.0           0.251747
              3.0           0.242326
              4.0           0.250017
              5.0           0.227481
              6.0           0.238148
              7.0           0.226576
              8.0           0.231321
              9.0           0.251369

In [21]:
n2[["inv_msa_match", "direct_cite", "year_diff"]].groupby(["inv_msa_match", "year_diff"]).mean()

direct_cite
inv_msa_match year_diff             
False         1.0           0.001850
              2.0           0.001875
              3.0           0.001046
              4.0           0.001411
              5.0           0.002168
              6.0           0.001997
              7.0           0.001455
              8.0           0.001269
              9.0           0.001227
True          1.0           0.016913
              2.0           0.017375
              3.0           0.016949
              4.0           0.012987
              5.0           0.014894
              6.0           0.016158
              7.0           0.006961
              8.0           0.006696
              9.0           0.012887

In [8]:
n2[["inv_msa_match", "direct_cite"]].groupby("inv_msa_match").mean()

,direct_cite
inv_msa_match,
False,0.002247
True,0.014929


In [20]:
n2[["bin_sim_docvecs", "inv_msa_match"]].groupby("bin_sim_docvecs").mean()

,inv_msa_match
bin_sim_docvecs,
-0.1,0.030809
0.0,0.036239
0.1,0.042733
0.2,0.051008
0.3,0.057132
0.4,0.071293
0.5,0.090346
0.6,0.104698
0.7,0.216080


In [27]:
n3 = n2.loc[n2["direct_cite"] == False]
n3[["bin_sim_docvecs", "inv_msa_match"]].groupby("bin_sim_docvecs").mean()

,inv_msa_match
bin_sim_docvecs,
-0.1,0.030809
0.0,0.035854
0.1,0.042004
0.2,0.050006
0.3,0.054918
0.4,0.068367
0.5,0.081421
0.6,0.083799
0.7,0.045752


In [28]:
n3[["inv_msa_match", "sim_docvecs", "year_diff"]].groupby(["inv_msa_match", "year_diff"]).mean()

sim_docvecs
inv_msa_match year_diff             
False         1.0           0.210937
              2.0           0.215979
              3.0           0.195966
              4.0           0.208792
              5.0           0.205426
              6.0           0.206703
              7.0           0.208531
              8.0           0.209074
              9.0           0.209683
True          1.0           0.243919
              2.0           0.250868
              3.0           0.241690
              4.0           0.249319
              5.0           0.224257
              6.0           0.236423
              7.0           0.226477
              8.0           0.230176
              9.0           0.250595

In [29]:
nt[["inv_msa_match", "sim_docvecs", "year_diff"]].groupby(["inv_msa_match", "year_diff"]).mean()

sim_docvecs
inv_msa_match year_diff             
False         1.0           0.129735
              2.0           0.129695
              3.0           0.126523
              4.0           0.133754
              5.0           0.124152
              6.0           0.128864
              7.0           0.131415
              8.0           0.122381
              9.0           0.122942
True          1.0           0.163369
              2.0           0.156970
              3.0           0.155776
              4.0           0.156201
              5.0           0.138380
              6.0           0.145314
              7.0           0.137311
              8.0           0.132130
              9.0           0.136622

In [30]:
nt.loc[nt["num_common_cites"] >= 1, ]

Index(['sim_docvecs', 'tp', 'op', 'term', 'sim_ldavecs', 'tp_appyear',
       'tp_inv_msa', 'tp_primclass', 'tp_naics_name', 'tp_title', 'op_appyear',
       'op_inv_msa', 'op_primclass', 'op_naics_name', 'op_title',
       'inv_msa_match', 'year_diff', 'direct_cite', 'num_common_cites',
       'norm_sim_ldavecs', 'norm_sim_docvecs', 'norm_num_common_cites',
       'common_pat_inv', 'bin_sim_docvecs'],
      dtype='object')